<a href="https://colab.research.google.com/github/wang201156/CFD_program/blob/main/Project_3_Two_Dimensional_Viscous_Scaler_Transport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### be discarded
import numpy as np
import matplotlib.pyplot as plt
import os

# === 參數設定 ===
use_coarse_grid = False  # 若為 True 則使用 coarse grid 驗證速度
# 設定網格參數

gamma=0
Nu=0.8
u=0.5
v=0.5

if use_coarse_grid:
    nx, ny = 81, 81
else:
    nx, ny = 321, 321


xmax, xmin = 4.0, 0.0
ymax, ymin = 4.0, 0.0
max_steps=40
tol = 1e-5
pi = np.pi

dx = (xmax - xmin) / (nx - 1)
dy = (ymax - ymin) / (ny - 1)
time=Nu /(u/dx + v/dy)

# 初始化矩陣
T =  np.zeros((nx, ny))
TEMP = np.zeros((nx, ny))
x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)
X, Y = np.meshgrid(x, y)

# 設定邊界條件
T[-1, :] = 0   # 底部邊界 T = 0
T[0, :] = 0  # 頂部邊界 T = 0
T[:, 0] = 0   # 左側邊界 T = 0
T[:, -1] = 0  # 右側邊界 T = 0
#=======================================================================================
# 設定初始條件
for i in range(nx):
    for j in range(ny):
      if 0.9 <= x[i] <= 1.1 and 0.9 <= y[j] <= 1.1:
        TEMP[i, j] = np.sin(5*pi*(x[i] - 0.9)) * np.sin(5*pi*(y[j] - 0.9))
T=TEMP.copy()
#=======================================================================================
# 迭代求解

RHS=np.zeros((nx, ny))
Tg = T.copy() # Evaluating residual
error = 1
count = 1
residuals = []

#neighbors coefficient and central coefficient.

aw = gamma*time / (2 * dx**2)
ae = gamma*time / (2 * dx**2)
as_ = gamma*time / (2 * dy**2)
an = gamma*time / (2 * dy**2)
ap = 1+ (aw+ae+as_+an)/2   ##  ap=1+aw+ae+as+an
au = u*time/dx
av = v*time/dy
# === 儲存圖像資料夾 ===
output_dir = "frames"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
#========================

for step in range(1, max_steps + 1):  #Tij=t_new

  residual = 1.0
  count = 0

  while residual > tol:
    T_new = T.copy()

    for i in range(1, nx - 1):
         for j in range(1, ny - 1):
           # convection term
           cnx=au/2* (T[i,j]-T[i-1,j]  + T_new[i,j]-T_new[i-1,j])
           cny=av/2* (T[i,j]-T[i,j-1]  + T_new[i,j]-T_new[i,j-1])
           # diffusion term(only n)
           diffx=( aw*T[i-1,j]-(aw+ae)*T[i,j]+ae*T[i+1,j] )/2
           diffy=( as_*T[i,j-1]-(as_+an)*T[i,j]+an*T[i,j+1] )/2

           RHS[i, j] = T[i,j] - (cnx+cny) + (diffx+diffy)
           T_new[i, j] = RHS[i,j] + (  aw*T_new[i-1, j] + ae*T_new[i+1, j] + as_*T_new[i, j-1] + an*T_new[i, j+1]  )/2
           T_new[i, j] = T_new[i, j]/ap
    residual = np.linalg.norm(T_new - T) / np.sqrt(nx * ny)
    T = T_new.copy()

  print(f"Step: {step}, Time: {step*time:.5f}, Residual: {residual:.2e}")

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# === 參數設定 ===
use_coarse_grid = False  # 若為 True 則使用 coarse grid 驗證速度

gamma = 0.001     # 設為 0 或 0.001
Nu = 0.8
u = 0.5
v = 0.5
max_steps = 400
tol = 1e-5
pi = np.pi

# === 空間格點設定 ===
if use_coarse_grid:
    nx, ny = 81, 81
else:
    nx, ny = 321, 321

xmax, xmin = 4.0, 0.0
ymax, ymin = 4.0, 0.0

dx = (xmax - xmin) / (nx - 1)
dy = (ymax - ymin) / (ny - 1)
dt = Nu / (u/dx + v/dy)

x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)
X, Y = np.meshgrid(x, y)

# === 初始條件設定 ===
T = np.zeros((nx, ny))
for i in range(nx):
    for j in range(ny):
        if 0.9 <= x[i] <= 1.1 and 0.9 <= y[j] <= 1.1:
            T[i, j] = np.sin(5 * pi * (x[i] - 0.9)) * np.sin(5 * pi * (y[j] - 0.9))

# === 系數設定 ===
aw = gamma*dt / (2 * dx**2)
ae = gamma*dt / (2 * dx**2)
as_ = gamma*dt / (2 * dy**2)
an = gamma*dt / (2 * dy**2)
ap = 1 + (aw + ae + as_ + an) / 2
au = u * dt / dx
av = v * dt / dy

# === 儲存圖像資料夾 ===
output_dir = "frames"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# === 時間迴圈 ===
for step in range(1, max_steps + 1):

    residual = 1.0
    T_new = T.copy()

    while residual > tol:
        T_old = T_new.copy()

        for i in range(1, nx - 1):
            for j in range(1, ny - 1):
                # convection term (upwind + Crank–Nicolson)
                cnx = au/2 * ((T[i, j] - T[i-1, j]) + (T_old[i, j] - T_old[i-1, j]))
                cny = av/2 * ((T[i, j] - T[i, j-1]) + (T_old[i, j] - T_old[i, j-1]))

                # diffusion term (central)
                diffx = (aw*T[i-1,j] - (aw+ae)*T[i,j] + ae*T[i+1,j]) / 2
                diffy = (as_*T[i,j-1] - (as_+an)*T[i,j] + an*T[i,j+1]) / 2

                RHS = T[i, j] - (cnx + cny) + (diffx + diffy)
                T_new[i, j] = RHS + aw*T_new[i-1, j] + ae*T_new[i+1, j] + as_*T_new[i, j-1] + an*T_new[i, j+1]
                T_new[i, j] = T_new[i, j] / ap


        residual = np.sum(np.abs(T_new - T_old)) / np.sqrt(nx * ny)  # Evaluating residual( residual normalizes for converge faster)

    T = T_new.copy()

    print(f"Step {step:4d}, Time = {step*dt:.4f}, Residual = {residual:.2e}")

    # 每 40 步畫一次圖
    if step % 40 == 0 or step == max_steps or step==1:
        plt.figure(figsize=(6,5))
        cp = plt.contourf(X, Y, T.T, 20, cmap='jet')
        plt.title(f'Temperature at step {step}')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.colorbar(cp)
        plt.savefig(f"{output_dir}/step_{step:04d}.png")
        plt.close()


Step    1, Time = 0.0100, Residual = 7.81e-06
Step    2, Time = 0.0200, Residual = 7.70e-06
Step    3, Time = 0.0300, Residual = 8.82e-06
Step    4, Time = 0.0400, Residual = 4.58e-06
Step    5, Time = 0.0500, Residual = 2.62e-06
Step    6, Time = 0.0600, Residual = 8.68e-06
Step    7, Time = 0.0700, Residual = 7.54e-06
Step    8, Time = 0.0800, Residual = 6.92e-06
Step    9, Time = 0.0900, Residual = 6.62e-06
Step   10, Time = 0.1000, Residual = 6.42e-06
Step   11, Time = 0.1100, Residual = 6.32e-06
Step   12, Time = 0.1200, Residual = 6.29e-06
Step   13, Time = 0.1300, Residual = 6.29e-06
Step   14, Time = 0.1400, Residual = 6.34e-06
Step   15, Time = 0.1500, Residual = 6.43e-06
Step   16, Time = 0.1600, Residual = 6.53e-06
Step   17, Time = 0.1700, Residual = 6.65e-06
Step   18, Time = 0.1800, Residual = 6.81e-06
Step   19, Time = 0.1900, Residual = 6.99e-06
Step   20, Time = 0.2000, Residual = 7.17e-06
Step   21, Time = 0.2100, Residual = 7.38e-06
Step   22, Time = 0.2200, Residual